In [1]:
import pandas as pd

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [3]:
base_path = '/media/alyaan/hdd/storage/truck_load/split details/data/'

In [4]:
prediction_path = "/home/alyaan/code/truck_load_classification/output"

In [5]:
test_path = os.path.join(base_path,"test.txt")
train_path = os.path.join(base_path,"train.txt")

In [6]:
with open(test_path) as f:
    test_split = f.readlines()

In [7]:
with open(train_path) as f:
    train_split = f.readlines()

In [8]:
test_file_names = [file_name.split("/")[-1].strip('\n').split(".")[0] for file_name in test_split]

In [9]:
train_file_names = [file_name.split("/")[-1].strip('\n').split(".")[0] for file_name in train_split]

In [10]:
test_file_df = pd.DataFrame(test_file_names,columns=["image_name"])
test_file_df['split'] = 'test'

In [11]:
train_file_df = pd.DataFrame(train_file_names,columns=["image_name"])
train_file_df['split'] = 'train'

In [12]:
train_test_combined = pd.concat([train_file_df,test_file_df])

In [13]:
train_test_combined.shape

(713, 2)

In [14]:
label_name_path = '/media/alyaan/hdd/storage/truck_load/split details/data/obj.names'

In [15]:
class_labels = pd.read_csv(label_name_path,names=['label_names'])
class_labels['gt_class_label'] = np.arange(len(class_labels))

In [16]:
class_labels

,label_names,gt_class_label
0,vehicle,0
1,vehicle_empty,1
2,vehicle_load,2


In [17]:
orig_image_width = 1920
orig_image_height = 1080

In [18]:
yolo_image_width = 608
yolo_image_height = 608

In [19]:
file_extension = [".txt",".jpeg"]

In [20]:
gt_class_name_label_path = "/media/alyaan/hdd/storage/truck_load/ground_truth/obj.names"

In [21]:
gt_annotatons_path = "/media/alyaan/hdd/storage/truck_load/ground_truth/annotations"

In [22]:
gt_file_name = os.listdir(gt_annotatons_path)
annotation_files =[file for file in gt_file_name if file.split(".")[1] == "txt"]

In [23]:
gt_col_names = ["gt_class_label","gt_xtopl","gt_ybottomr","gt_width","gt_height"]

In [24]:
full_df_col_names = ["image_name"] + gt_col_names

In [25]:
gt_full_df = pd.DataFrame(columns=full_df_col_names)
for file in annotation_files:
    file_path = os.path.join(gt_annotatons_path,file)
    gt_image_annotations = pd.read_csv(file_path,delim_whitespace=True,header=None,names=gt_col_names)
    gt_image_annotations['image_name'] = file.split(".")[0]
    gt_full_df = gt_full_df.append(gt_image_annotations)

In [26]:
gt_full_labels_df = pd.merge(gt_full_df,train_test_combined,how='inner',on='image_name')

In [27]:
gt_full_labels_df.columns

Index(['image_name', 'gt_class_label', 'gt_xtopl', 'gt_ybottomr', 'gt_width',
       'gt_height', 'split'],
      dtype='object')

In [28]:
gt_full_labels_df.query('split == \'test\'').head(2)

,image_name,gt_class_label,gt_xtopl,gt_ybottomr,gt_width,gt_height,split
20,1617858725292,0,0.50063,0.575500,0.99874,0.599519,test
21,1617858725292,2,0.45825,0.546722,0.91650,0.230222,test


In [29]:
gt_full_labels_df.query('split == \'train\'').head(3)

,image_name,gt_class_label,gt_xtopl,gt_ybottomr,gt_width,gt_height,split
0,1617593545228,0,0.866862,0.549722,0.266276,0.557556,train
1,1617593545228,2,0.897279,0.417824,0.204359,0.195444,train
2,1619846400774,0,0.868547,0.570704,0.262906,0.601926,train


In [30]:
gt_full_labels_df[gt_full_labels_df.split == 'test']

,image_name,gt_class_label,gt_xtopl,gt_ybottomr,gt_width,gt_height,split
20,1617858725292,0,0.500630,0.575500,0.998740,0.599519,test
21,1617858725292,2,0.458250,0.546722,0.916500,0.230222,test
56,1617940655557,0,0.863826,0.546722,0.272349,0.558759,test
57,1617940655557,2,0.891479,0.398042,0.217042,0.215824,test
64,1617942939123,0,0.883385,0.603079,0.233229,0.460435,test
...,...,...,...,...,...,...,...
689,1617864027440,2,0.900922,0.414829,0.198156,0.215824,test
696,1617858078801,0,0.500000,0.559912,1.000000,0.568343,test
697,1617858078801,2,0.455552,0.592306,0.911104,0.117556,test
720,1617854864251,0,0.855732,0.544324,0.288536,0.580333,test


In [31]:
gt_full_labels_df = pd.merge(gt_full_labels_df,class_labels,how='left',on='gt_class_label')

In [32]:
gt_full_labels_df.head(3)

,image_name,gt_class_label,gt_xtopl,gt_ybottomr,gt_width,gt_height,split,label_names
0,1617593545228,0,0.866862,0.549722,0.266276,0.557556,train,vehicle
1,1617593545228,2,0.897279,0.417824,0.204359,0.195444,train,vehicle_load
2,1619846400774,0,0.868547,0.570704,0.262906,0.601926,train,vehicle


In [52]:
pred_col_names = ["pred_class_label","pred_xtopl","pred_ybottomr","pred_width","pred_height","confidence"]

In [53]:
prediction_filename = os.listdir(prediction_path)

In [54]:
pred_full_df = pd.DataFrame(columns=pred_col_names)
for file in prediction_filename:
    file_path = os.path.join(prediction_path,file)
    prediction_text = pd.read_csv(file_path,delim_whitespace=True,header=None,names=pred_col_names)
    prediction_text['image_name'] = file.split(".")[0]
    pred_full_df = pred_full_df.append(prediction_text)

In [55]:
pred_full_df.head()

,pred_class_label,pred_xtopl,pred_ybottomr,pred_width,pred_height,confidence,image_name
0,2,0.8962,0.4135,0.2089,0.2252,99.18,1619071431920
1,0,0.8609,0.5491,0.2797,0.5884,99.90,1619071431920
0,2,0.9030,0.4270,0.1982,0.2149,99.68,1618715170454
1,0,0.8713,0.5509,0.2544,0.5587,99.85,1618715170454
0,0,0.7669,0.5398,0.4806,0.4667,96.69,1617086814988


In [56]:
pred_full_df.shape

(737, 7)

In [ ]:
len(prediction_filename)

In [ ]:
prediction_filename[0:2]

In [ ]:
len(test_file_names)

In [ ]:
test_file_names[0:2]

In [ ]:
len(set(test_file_names).intersection(set(prediction_filename)))